In [6]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox as tkMsgBox
from tkinter import filedialog
from tkinter import Tk, Label, Button, OptionMenu, StringVar
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
from datetime import datetime
from IPython.display import display, clear_output, HTML

In [7]:
class SalesDataAnalysis:
    
    def __init__(self):
        self.df = pd.read_csv('sales_data.csv')
        self.clean_data()
        
    def clean_data(self):
        # Strip column names of leading and trailing whitespaces
        self.df.columns = self.df.columns.str.strip()

        # Remove whitespaces from cell values
        self.df = self.df.applymap(lambda x: x.strip() if isinstance(x,str) else x)

        # More data cleaning
        self.df = self.df.applymap(lambda x: x.replace('$', '').replace(',', '').replace('-', '0') if isinstance(x, str) and '$' in x else x)

        # Add $ to the names of columns that represent amounts of money
        list_add_currency = ['Manufacturing Price', 'Sale Price', 'Gross Sales', 'Discounts', 'Sales', 'COGS', 'Profit']
        dict_add_currency = {col:col+'($)' for col in list_add_currency}
        self.df.columns = [dict_add_currency[col] if col in dict_add_currency else col for col in self.df.columns]

        # For negative values, remove the parenthesis and add a minus sign
        self.df = self.df.applymap(lambda x: '-'+x[1:-1] if isinstance(x,str) and x.startswith('(') and x.endswith(')') else x)

        # Align datetime to pandas format
        self.df["DateTime"] = pd.to_datetime(self.df["DateTime"])

        # Create the Year, Year-Month, MonthNum, and Month columns from the DateTime column
        self.df['Year'] = self.df["DateTime"].dt.year
        self.df["Year-Month"] = self.df["DateTime"].dt.strftime('%Y-%m')
        self.df['MonthNum'] = self.df["DateTime"].dt.month
        self.df['Month'] = self.df["DateTime"].dt.strftime('%B')

        # Set the columns to the correct data type
        dict_dtype = {"Segment": str, "Country":str, "Product":str, "Discount Band":str, "Units Sold":float, "Manufacturing Price($)":float,"Sale Price($)":float, "Gross Sales($)":float, "Discounts($)":float, "Sales($)":float, "COGS($)":float, "Profit($)":float, "Year":str, "DateTime":str, "MonthNum":str, "Month":str}
        self.df = self.df.astype(dict_dtype)
        
               
        
        #1 Summary statistics
    def summary_statistics(self):
        # Define parameters for table styles
        cell_hover = {  # for row hover use <tr> instead of <td>
            'selector': 'td:hover',
            'props': [('background-color', '#848080')]
        }
        index_names = {
            'selector': '.index_name',
            'props': 'font-style: italic; color: darkgrey; font-weight:normal;'
        }
        headers = {
            'selector': 'th:not(.index_name)',
            'props': 'background-color: #001746; color: white;'
        }
        s = self.df.select_dtypes(include=['number']).describe().style.set_table_styles([cell_hover, index_names, headers])\
        .background_gradient(axis=None, vmin=1, vmax=5, cmap="GnBu")
        display(s)

        #2 Histograms
    def histograms(self):
        x_list = [{"Units Sold":"Segment"}, {"Units Sold":"Discount Band"}, {"Units Sold":"Year"}]
        for item in x_list:
            for k,v in item.items():
                plt = px.histogram(self.df,x=k, nbins=25, color=v).update_layout(
                title={
                    'text':f"Histogram of {k} colored by {v}",
                    'x':0.5,
                    'y':0.93,
                    'xanchor':'center',
                    'yanchor':'top'
                },
                title_font_size=22)
                plt.show()
        
        #3 Boxplots
    def boxplots(self):
        s = px.box(self.df,x="Country",y="Units Sold",color="Year").update_layout(title={
                'text':"Boxplots of Units Sold",
                'x':0.5,
                'y':0.93,
                'xanchor':'center',
                'yanchor':'top'
            },title_font_size=24)
        display(s)
        
        #4 Correlation Matrix
        
        # Define function for conditional formatting
    def corr_matrix(self):
        headers = {'selector': 'th:not(.index_name)','props': 'background-color: #001746; color: white;'}
        def conditional_formatting(value):
            if abs(value) == 1:
                return 'background-color: #082604'
            if abs(value) > 0.9:
                return 'background-color: #006400'
            elif abs(value) > 0.7:
                return 'background-color: #38b000'
            elif abs(value) > 0.5:
                return 'background-color: #F8F808'
            elif abs(value) > 0.3:
                return 'background-color: #CAF2E3'
            elif abs(value) > 0.1:
                return 'background-color: #F8961E'
            elif abs(value) <= 0.1:
                return 'background-color: #FF0000'
            
        s = self.df[['Units Sold','Manufacturing Price($)', 'Sale Price($)', 'Gross Sales($)','Discounts($)', 'Sales($)', 
                'COGS($)', 'Profit($)']].corr().style.applymap(conditional_formatting).set_table_styles([headers])
        
        lst = ['Units Sold','Manufacturing Price(\$)', 'Sale Price(\$)', 'Gross Sales(\$)','Discounts(\$)', 'Sales(\$)', 'COGS(\$)', 'Profit(\$)']
        comb_list = list(f'{i} | {j}' for i in lst for j in lst)
        tt_list = [comb_list[i:i+8] for i in range(0, len(comb_list), 8)]
        
        tooltips = pd.DataFrame(tt_list,index=pd.Index(['Units Sold','Manufacturing Price($)', 'Sale Price($)', 
                                                        'Gross Sales($)','Discounts($)', 'Sales($)', 'COGS($)', 
                                                        'Profit($)']),columns=self.df.columns[4:12])
        display(s.set_tooltips(tooltips, props='visibility: hidden; position: absolute; z-index: 1; border: 1px solid #000086;'
                         'background-color: #CECECE; color: #000076; font-size: 0.8em;'
                         'transform: translate(0px, -24px); padding: 0.6em; border-radius: 0.5em;'))
        

    
        ##1 Scatterplots
    def scatterplots(self):
        s = px.scatter(self.df, x="Discounts($)", y="Sales($)", color="Discount Band").update_layout(title={
                'text':f"Scatterplot of Sales vs Discounts",
                'x':0.5,
                'y':0.93,
                'xanchor':'center',
                'yanchor':'top'},title_font_size=24)
        display(s)
        
        ##2 Barcharts
    
        # Total Sales by Product by Country
    def barcharts_1(self):
        fig1 = px.histogram(self.df, y='Product', x='Sales($)', color='Country',pattern_shape="Country", pattern_shape_sequence=[".","\\", "x", "+","|"],
                          labels={
                              'sum of Sales($)':'Sales'
                          }).\
        update_layout(yaxis={'categoryorder':'total ascending'},xaxis_title="Sales",title={
                'text':'Total Sales by Product by Country',
                'x':0.5,
                'y':0.93,
                'xanchor':'center',
                'yanchor':'top'},title_font_size=20)
        fig1.show()
        
        # Total Sales by Country by Year
    def barcharts_2(self):
        fig2 = px.histogram(self.df, x='Country', y='Sales($)', color='Year', barmode='group',pattern_shape="Year", pattern_shape_sequence=[".","\\", "x"]).\
        update_layout(yaxis={'categoryorder':'total descending'},yaxis_title="Sales",title={
                'text':'Total Sales by Country by Year',
                'x':0.5,
                'y':0.93,
                'xanchor':'center',
                'yanchor':'top'},title_font_size=20)
        fig2.show()

        ##3 Pie charts

        # Total Sales by Segment
    def piechart_1(self):
        fig3 = go.Figure(data=[go.Pie(labels=self.df["Segment"], values=self.df["Sales($)"], hole=.5)]).\
        update_layout(title={
                'text':'Total Sales by Segment',
                'x':0.5,
                'y':0.93,
                'xanchor':'center',
                'yanchor':'top'},title_font_size=20)
        fig3.show()
        
        # Total Discounts by Country
    def piechart_2(self):
        fig4 = px.pie(self.df, values='Discounts($)', names='Country').\
        update_layout(title={
                'text':'Discounts by Country',
                'x':0.5,
                'y':0.99,
                'xanchor':'center',
                'yanchor':'top'},title_font_size=20)
        fig4.show()
        
        ### Time series charts

        ##4 Area chart
    def areachart(self):
        area_df = pd.pivot_table(self.df, values="Sales($)", index=["Year-Month"], columns=["Country"])
        s = px.area(area_df).\
        update_layout(yaxis_title="Sales",title={
                'text':'Total Sales by Country',
                'x':0.5,
                'y':0.93,
                'xanchor':'center',
                'yanchor':'top'}, title_font_size=20)
        display(s)
        
        
       ##5 Line chart
    def linechart(self):
        line_df = self.df.groupby('Year-Month').sum(numeric_only=True)["Sales($)"]
        s = px.line(line_df).\
        update_layout(yaxis_title="Sales",showlegend=False,title={
                'text':'Total Sales',
                'x':0.5,
                'y':0.93,
                'xanchor':'center',
                'yanchor':'top'},title_font_size=20) 
        display(s)


In [8]:
class GUIApp:
    
    def __init__(self, main_window, data_file):
        self.main_window = main_window
        self.main_window.title("Data Analysis and Visualization")
        self.main_window.geometry("300x200")

        self.style = ttk.Style()
#         self.style.theme_use('clam')
        
        self.main_window.tk.call("source","azure.tcl")
        self.main_window.tk.call("set_theme","dark")
        self.style.configure("TCombobox", fieldbackground="blue", background="white", arrowcolor="blue")

        self.sales_analysis = SalesDataAnalysis()

        # EDA and VIZ Radio Buttons
        self.analysis_type = tk.StringVar(value='EDA')
        self.EDA_radio = ttk.Radiobutton(self.main_window, text='EDA', variable=self.analysis_type, value='EDA', command=self.toggle_options)
        self.VIZ_radio = ttk.Radiobutton(self.main_window, text='VIZ', variable=self.analysis_type, value='VIZ', command=self.toggle_options)
        self.EDA_radio.pack()
        self.VIZ_radio.pack()

        # Dropdown Menus
        self.EDA_options = ttk.Combobox(self.main_window, values=["Summary Statistics", "Histograms", "Boxplots", "Correlation Matrix"], state='readonly')
        self.VIZ_options = ttk.Combobox(self.main_window, values=["Scatterplot", "Bar-Sales by Product", "Bar-Sales by Country", "Pie-Sales by Segment", "Pie-Discounts by Country", "Area Chart", "Line Chart"], state='disabled')
        self.EDA_options.pack()
        self.VIZ_options.pack()

        # Canvas
        self.fig_canvas = FigureCanvasTkAgg(Figure(figsize=(5, 4)), master=self.main_window)
        self.fig_canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

        # Plot Button
        self.plot_button = tk.Button(self.main_window, text='Plot', width=20, bg="#00005C", fg="white", activebackground="#7979FF", command=self.plot_chart)
        self.plot_button.pack(pady=2)
        
        self.quit_button = tk.Button(self.main_window, text='Exit', width=10, bg="#9E0000", fg="white", activebackground="#FF9F9F", command=self.main_window.quit)
        self.quit_button.pack(pady=1)

    def toggle_options(self):
        if self.analysis_type.get() == 'EDA':
            self.EDA_options.config(state='readonly')
            self.VIZ_options.config(state='disabled')
        else:
            self.VIZ_options.config(state='readonly')
            self.EDA_options.config(state='disabled')

    def plot_chart(self):
        chosen_option = self.EDA_options.get() if self.analysis_type.get() == 'EDA' else self.VIZ_options.get()
        if chosen_option:
            # Clear the previous figure
            self.fig_canvas.figure.clf()
            
            if chosen_option == "Summary Statistics":
                clear_output(wait=True)
                self.sales_analysis.summary_statistics()

            elif chosen_option == "Histograms":
                clear_output(wait=True)
                sales_analysis.histograms()
            
            elif chosen_option == "Boxplots":
                clear_output(wait=True)
                self.sales_analysis.boxplots()
            
            elif chosen_option == "Correlation Matrix":
                clear_output(wait=True)
                self.sales_analysis.corr_matrix()

            elif chosen_option == "Scatterplot":
                clear_output(wait=True)
                self.sales_analysis.scatterplots()

            elif chosen_option == "Bar-Sales by Product":
                clear_output(wait=True)
                self.sales_analysis.barcharts_1()
            
            elif chosen_option == "Bar-Sales by Country":
                clear_output(wait=True)
                self.sales_analysis.barcharts_2()
            
            elif chosen_option == "Pie-Sales by Segment":
                clear_output(wait=True)
                self.sales_analysis.piechart_1()
            
            elif chosen_option == "Pie-Discounts by Country":
                clear_output(wait=True)
                self.sales_analysis.piechart_2()
            
            elif chosen_option == "Area Chart":
                clear_output(wait=True)
                self.sales_analysis.areachart()
            
            elif chosen_option == "Line Chart":
                clear_output(wait=True)
                self.sales_analysis.linechart()
            else:
                messagebox.showerror("Error", "Invalid chart type selected")

In [9]:
# Increase the height of the display
display(HTML("<style>div.output_scroll { height: 65em; }</style>"))

In [10]:
if __name__ == "__main__":
    
    sales_analysis = SalesDataAnalysis()

    main_window = tk.Tk()
    gui_app = GUIApp(main_window, sales_analysis)
    main_window.mainloop()